# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B 를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A:산, B:바다, C:나무, D:물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ? ] <br>
모델이 예측한 가장 적합한 단어: 잠입 <br>
당신의 답변과 모델 예측의 유사도: 0.34 <br>
아쉽네요. 더 생각해보세요.

In [1]:
# !pip install Korpora


   ------------- -------------------------- 1/3 [xlrd]
   -------------------------- ------------- 2/3 [Korpora]
   ---------------------------------------- 3/3 [Korpora]



In [1]:
from Korpora import Korpora
# Korpora.fetch('korean_petitions')

corpus = Korpora.load('korean_petitions')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-09
[Korpora] Corpus `korean_petitions` is already installed at C:

In [2]:
from gensim.models import FastText, Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import kss
from konlpy.tag import Okt
import string
import re
import random

In [3]:
corpus

In [20]:
sen = list(corpus.train.texts)[:50000] # + corpus.train.pairs) + list(corpus.dev.texts + corpus.dev.pairs) + list(corpus.test.texts + corpus.test.pairs)
len(sen)

50000

In [5]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    
    return stopwords

In [6]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
okt = Okt()
preprocessed_sentences = []
ko_stopwords = load_stopwords('ko_stopwords.txt')

for s in sen:
    s = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', s)
    # tokens = okt.morphs(s)
    tokens = okt.nouns(s)
    cleaned_tokens = [token for token in tokens if token not in ko_stopwords and token not in punct]

    preprocessed_sentences.append(cleaned_tokens)

len(preprocessed_sentences)

50000

### 오류!

**sentences에는 각 sentence별로 나눠진 list를 넣어야됨 2차원 배열**

In [22]:
fasttext_model = FastText(
    sentences=preprocessed_sentences,
    vector_size=100,
    window=5,
    min_count=3,
    sg=1
)
# vector_size 조정
# window 크기 조정
# sg 1로 바꿔보기

In [23]:
len(fasttext_model.wv.index_to_key)

23585

In [33]:
fasttext_model.wv.save_word2vec_format('game_w2v')

In [37]:
A = random.choice(fasttext_model.wv.index_to_key)
# print(A)
threshold = 0.8
filtered = []
while not filtered:
    filtered = [word for word, score in fasttext_model.wv.most_similar(A, topn=1000) if score >= threshold]
    threshold -= 0.1

B = random.choice(filtered)
sim_ab = fasttext_model.wv.similarity(A, B)

C = A
filtered = []
while (C == A or C == B) and (not filtered):
    C = random.choice(fasttext_model.wv.index_to_key)
    filtered = [word for word, score in fasttext_model.wv.most_similar(C, topn=1000) if score >= threshold]

print(f"관계 [ {A} : {B} = {C} : ? ]")

D = input()

sim_user = fasttext_model.wv.similarity(C, D)
model_out = ''
sim_model = 0
for model_choice in fasttext_model.wv.most_similar(C, topn=1000):
    if abs(model_choice[1]-sim_ab) < abs(sim_model - sim_ab):
        sim_model = model_choice[1]
        model_out = model_choice[0]

print(f"모델이 예측한 단어 : {model_out}")
print(f"입력한 답: {D}")
print(f"당신의 답변과 모델 예측의 유사도: {fasttext_model.wv.similarity(D, model_out)}")


관계 [ 장년 : 빈곤층 = 자치구 : ? ]
모델이 예측한 단어 : 특별자치도
입력한 답: 강서구
당신의 답변과 모델 예측의 유사도: 0.5835752487182617


In [38]:
print(f"{A}와 {B}의 유사도: {sim_ab}")
print(f"{C}와 {model_out}의 유사도: {sim_model}")
print(f"{C}와 {D}의 유사도: {sim_user}")
print(fasttext_model.wv.most_similar(C))

장년와 빈곤층의 유사도: 0.8370791673660278
자치구와 특별자치도의 유사도: 0.8214325308799744
자치구와 강서구의 유사도: 0.6568778157234192
[('특별자치도', 0.8214325308799744), ('광산구', 0.8176490664482117), ('대덕', 0.817389190196991), ('공진', 0.8127878308296204), ('동래구', 0.8121123909950256), ('김제시', 0.8113343119621277), ('약대동', 0.8104628920555115), ('산림조합', 0.8104188442230225), ('유성구', 0.8060669302940369), ('덕진구', 0.8054627180099487)]
